# Complete Backtest Analysis with Pyfolio

This notebook demonstrates best practices for analyzing Zipline backtest results using pyfolio-reloaded.

## Key Analysis Components:
1. **Returns Analysis** - Daily/cumulative returns, drawdowns
2. **Risk Metrics** - Sharpe, Sortino, Max Drawdown, Volatility
3. **Position Analysis** - Holdings over time, turnover
4. **Transaction Analysis** - Trade timing and costs
5. **Factor Analysis** - Exposure to market factors (optional)

In [ ]:
# Install pyfolio-reloaded if not already installed
# !pip install pyfolio-reloaded

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

from zipline import run_algorithm
from zipline.api import (
    order_target_percent,
    symbol,
    record,
    schedule_function,
    date_rules,
    time_rules
)

try:
    import pyfolio as pf
except ImportError:
    print("Installing pyfolio-reloaded...")
    !pip install pyfolio-reloaded
    import pyfolio as pf

%matplotlib inline

## Step 1: Run Your Backtest

Define and run your trading strategy:

In [ ]:
def initialize(context):
    """
    Initialize strategy - runs once at start
    """
    # Define universe
    context.stocks = [
        symbol('AAPL'),
        symbol('MSFT'),
        symbol('GOOGL')
    ]
    
    # Schedule rebalancing
    schedule_function(
        rebalance,
        date_rules.month_start(),
        time_rules.market_open()
    )
    
def rebalance(context, data):
    """
    Rebalance portfolio - equal weight strategy
    """
    weight = 1.0 / len(context.stocks)
    
    for stock in context.stocks:
        if data.can_trade(stock):
            order_target_percent(stock, weight)
            
def analyze(context, perf):
    """
    Analyze results - runs once at end
    """
    # Store performance data for later analysis
    context.perf = perf

# Run the backtest
print("Running backtest...")
results = run_algorithm(
    start=pd.Timestamp('2023-01-01'),
    end=pd.Timestamp('2024-12-31'),
    initialize=initialize,
    analyze=analyze,
    capital_base=100000,
    bundle='sharadar'
)

print("\n✓ Backtest complete!")
print(f"Total Return: {(results['returns'] + 1).prod() - 1:.2%}")

## Step 2: Extract Data for Pyfolio

Convert Zipline results into pyfolio format:

In [ ]:
# Extract returns, positions, and transactions from Zipline results
returns, positions, transactions = pf.utils.extract_rets_pos_txn_from_zipline(results)

print("Extracted data:")
print(f"Returns: {len(returns)} days")
print(f"Positions: {len(positions)} days × {len(positions.columns)} assets")
print(f"Transactions: {len(transactions)} trades")
print("\nReturns sample:")
print(returns.head())

## Step 3: Create Full Tear Sheet

Generate comprehensive performance analysis:

In [ ]:
# Create full tear sheet with all analysis
pf.create_full_tear_sheet(
    returns,
    positions=positions,
    transactions=transactions,
    live_start_date=None,  # Set if you have live trading data
    round_trips=True  # Analyze round-trip trades
)

## Step 4: Key Metrics Summary

Extract and display important metrics:

In [ ]:
# Calculate key performance metrics
metrics = {
    'Total Return': pf.timeseries.cum_returns_final(returns),
    'Annual Return': pf.timeseries.annual_return(returns),
    'Annual Volatility': pf.timeseries.annual_volatility(returns),
    'Sharpe Ratio': pf.timeseries.sharpe_ratio(returns),
    'Sortino Ratio': pf.timeseries.sortino_ratio(returns),
    'Max Drawdown': pf.timeseries.max_drawdown(returns),
    'Calmar Ratio': pf.timeseries.calmar_ratio(returns),
    'Omega Ratio': pf.timeseries.omega_ratio(returns),
    'Stability': pf.timeseries.stability_of_timeseries(returns),
}

# Display metrics
print("\n" + "="*50)
print("KEY PERFORMANCE METRICS")
print("="*50)
for metric, value in metrics.items():
    if 'Return' in metric or 'Drawdown' in metric or 'Volatility' in metric:
        print(f"{metric:.<30} {value:.2%}")
    else:
        print(f"{metric:.<30} {value:.3f}")
print("="*50)

## Step 5: Individual Analysis Components

Create specific tear sheets for detailed analysis:

In [ ]:
# Returns tear sheet - focused on return characteristics
pf.create_returns_tear_sheet(returns)

In [ ]:
# Position analysis - how holdings evolved over time
pf.create_position_tear_sheet(returns, positions)

In [ ]:
# Transaction analysis - trade timing and costs
pf.create_txn_tear_sheet(returns, positions, transactions)

In [ ]:
# Round trip analysis - analyze complete buy-sell cycles
pf.create_round_trip_tear_sheet(returns, positions, transactions)

## Step 6: Rolling Performance Analysis

Analyze how performance metrics change over time:

In [ ]:
# Rolling Sharpe ratio (6-month window)
rolling_sharpe = returns.rolling(126).apply(lambda x: pf.timeseries.sharpe_ratio(x))

fig, ax = plt.subplots(figsize=(14, 6))
rolling_sharpe.plot(ax=ax, label='6-Month Rolling Sharpe')
ax.axhline(0, color='black', linestyle='--', linewidth=0.5)
ax.axhline(1, color='green', linestyle='--', linewidth=0.5, alpha=0.5)
ax.set_title('Rolling Sharpe Ratio (6-Month Window)', fontsize=14, fontweight='bold')
ax.set_ylabel('Sharpe Ratio')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Average Rolling Sharpe: {rolling_sharpe.mean():.3f}")
print(f"Sharpe Volatility: {rolling_sharpe.std():.3f}")

## Step 7: Drawdown Analysis

Detailed analysis of underwater periods:

In [ ]:
# Plot underwater (drawdown) chart
fig, ax = plt.subplots(figsize=(14, 6))
pf.plot_drawdown_underwater(returns, ax=ax)
plt.tight_layout()
plt.show()

# Get top 5 drawdown periods
drawdown_periods = pf.timeseries.gen_drawdown_table(returns, top=5)
print("\nTop 5 Drawdown Periods:")
print(drawdown_periods)

## Step 8: Save Results

Export data for further analysis:

In [ ]:
# Save to pickle for later use
results.to_pickle('backtest_results.pkl')
print("✓ Saved results to backtest_results.pkl")

# Export metrics to CSV
metrics_df = pd.DataFrame([metrics])
metrics_df.to_csv('backtest_metrics.csv', index=False)
print("✓ Saved metrics to backtest_metrics.csv")

# Export returns for external analysis
returns.to_csv('backtest_returns.csv')
print("✓ Saved returns to backtest_returns.csv")

## Best Practices Summary

### 1. **Avoid Overfitting**
- Don't optimize parameters based on backtest results
- Use out-of-sample testing
- Test across different market regimes

### 2. **Realistic Simulation**
- Account for transaction costs
- Include slippage models
- Consider market impact

### 3. **Multiple Timeframes**
- Test on different historical periods
- Analyze rolling performance windows
- Check consistency of Sharpe ratio

### 4. **Risk Analysis**
- Focus on drawdowns, not just returns
- Analyze underwater periods
- Monitor position concentration

### 5. **Transaction Analysis**
- Review turnover rates
- Analyze trade profitability
- Check holding periods

### Key Metrics to Monitor:
- **Sharpe Ratio** > 1.0 (good), > 2.0 (excellent)
- **Max Drawdown** < 20% (acceptable for most strategies)
- **Calmar Ratio** (Return / Max Drawdown) > 0.5
- **Win Rate** > 50% for mean-reversion, varies for trend-following
- **Profit Factor** > 1.5 (gross profit / gross loss)